In [2]:
pip install xgboost

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance

In [3]:
data = pd.read_csv("AIML Dataset.csv")

In [4]:
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [5]:
data = data.rename(columns = {'oldbalanceOrg': 'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', 
                                    'oldbalanceDest':'oldBalanceDest', 'newbalanceDest' : 'newBalanceDest'})

In [6]:
from collections import Counter

type_list = list(data.loc[data.isFraud == 1].step.values)
type_counted_list = Counter(type_list)
type_counted_list.most_common(20)

[(212, 40),
 (523, 30),
 (249, 28),
 (387, 28),
 (425, 28),
 (501, 28),
 (730, 28),
 (160, 26),
 (398, 26),
 (66, 24),
 (406, 24),
 (625, 24),
 (694, 24),
 (22, 23),
 (6, 22),
 (34, 22),
 (74, 22),
 (149, 22),
 (250, 22),
 (262, 22)]

In [7]:
amount_list = list(data.loc[data.isFraud == 1].amount.values)

amount_counted_list = Counter(amount_list)
amount_counted_list.most_common(20)

[(10000000.0, 287),
 (0.0, 16),
 (429257.45, 4),
 (1165187.89, 4),
 (181.0, 2),
 (2806.0, 2),
 (20128.0, 2),
 (1277212.77, 2),
 (35063.63, 2),
 (25071.46, 2),
 (235238.66, 2),
 (1096187.24, 2),
 (963532.14, 2),
 (14949.84, 2),
 (18627.02, 2),
 (10539.37, 2),
 (22877.0, 2),
 (2930418.44, 2),
 (169941.73, 2),
 (13707.11, 2)]

In [8]:
nameOrig_list = list(data.loc[data.isFraud == 1].nameOrig.values)

nameOrig_counted_list = Counter(nameOrig_list)
nameOrig_counted_list.most_common(20)

[('C1305486145', 1),
 ('C840083671', 1),
 ('C1420196421', 1),
 ('C2101527076', 1),
 ('C137533655', 1),
 ('C1118430673', 1),
 ('C749981943', 1),
 ('C1334405552', 1),
 ('C467632528', 1),
 ('C1364127192', 1),
 ('C1635772897', 1),
 ('C669700766', 1),
 ('C1275464847', 1),
 ('C13692003', 1),
 ('C1872047468', 1),
 ('C1499825229', 1),
 ('C1093223281', 1),
 ('C77163673', 1),
 ('C1440057381', 1),
 ('C430329518', 1)]

In [9]:
oldBalanceOrig_list = list(data.loc[data.isFraud == 1].oldBalanceOrig.values)

oldBalanceOrig_counted_list = Counter(oldBalanceOrig_list)
oldBalanceOrig_counted_list.most_common(20)

[(10000000.0, 142),
 (0.0, 41),
 (429257.45, 4),
 (1165187.89, 4),
 (181.0, 2),
 (2806.0, 2),
 (20128.0, 2),
 (1277212.77, 2),
 (35063.63, 2),
 (25071.46, 2),
 (235238.66, 2),
 (1096187.24, 2),
 (963532.14, 2),
 (14949.84, 2),
 (18627.02, 2),
 (10539.37, 2),
 (22877.0, 2),
 (2930418.44, 2),
 (169941.73, 2),
 (13707.11, 2)]

In [10]:
type_list = list(data.loc[data.isFraud == 1].type.values)

type_counted_list = Counter(type_list)
type_counted_list.most_common(20)

[('CASH_OUT', 4116), ('TRANSFER', 4097)]

In [11]:
isFlaggedFraud_list = list(data.loc[data.isFraud == 1].isFlaggedFraud.values)

isFlaggedFraud_counted_list = Counter(isFlaggedFraud_list)
isFlaggedFraud_counted_list.most_common(20)

[(0, 8197), (1, 16)]

In [12]:
X = data.loc[(data.type == 'CASH_OUT') | (data.type == 'TRANSFER')]
randomState = 5
np.random.seed(randomState)
Y = X['isFraud']
del X['isFraud']


del X['nameDest'] 
del X['nameOrig']
del X['isFlaggedFraud']

In [13]:
X.head()


,step,type,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest
2,1,TRANSFER,181.00,181.0,0.0,0.0,0.00
3,1,CASH_OUT,181.00,181.0,0.0,21182.0,0.00
15,1,CASH_OUT,229133.94,15325.0,0.0,5083.0,51513.44
19,1,TRANSFER,215310.30,705.0,0.0,22425.0,0.00
24,1,TRANSFER,311685.89,10835.0,0.0,6267.0,2719172.89


In [14]:
X_fraud = X.loc[Y==1]
X_nonFraud = X.loc[Y==0]

In [15]:
X.loc[X.type == 'TRANSFER', 'type'] = 0
X.loc[X.type == 'CASH_OUT', 'type'] = 1

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [25]:
data.info()
data.drop(columns = ['type','nameOrig','nameDest'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldBalanceOrig  float64
 5   newBalanceOrig  float64
 6   nameDest        object 
 7   oldBalanceDest  float64
 8   newBalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


,step,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
0,1,9839.64,170136.00,160296.36,0.00,0.00,0,0
1,1,1864.28,21249.00,19384.72,0.00,0.00,0,0
2,1,181.00,181.00,0.00,0.00,0.00,1,0
3,1,181.00,181.00,0.00,21182.00,0.00,1,0
4,1,11668.14,41554.00,29885.86,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...
6362615,743,339682.13,339682.13,0.00,0.00,339682.13,1,0
6362616,743,6311409.28,6311409.28,0.00,0.00,0.00,1,0
6362617,743,6311409.28,6311409.28,0.00,68488.84,6379898.11,1,0
6362618,743,850002.52,850002.52,0.00,0.00,0.00,1,0


In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = randomState)
weights = (Y==0).sum()/(1.0 *  (Y==1).sum())
xgb_classifier1 = XGBClassifier(max_depth = 3, scale_pos_weight = weights, n_jobs = 4)
xgb_prediction1 = xgb_classifier1.fit(X_train, Y_train).predict_proba(X_test)

print('AUPRC = {}'.format(average_precision_score(Y_test, xgb_prediction1[:,1])))

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


ValueError: DataFrame.dtypes for data must be int, float, bool or category.  When
categorical type is supplied, DMatrix parameter `enable_categorical` must
be set to `True`. Invalid columns:type

In [23]:
data

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = randomState)

weights = (Y==0).sum()/(1.0 *  (Y==1).sum())
xgb_classifier3 = XGBClassifier(max_depth = 3, scale_pos_weight = weights, n_jobs = 4)
xgb_prediction3 = xgb_classifier3.fit(X_train, Y_train).predict_proba(X_test)

print('AUPRC = {}'.format(average_precision_score(Y_test, xgb_prediction3[:,1])))

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


ValueError: DataFrame.dtypes for data must be int, float, bool or category.  When
categorical type is supplied, DMatrix parameter `enable_categorical` must
be set to `True`. Invalid columns:type